In [1]:
# After we have successfully calibrated the central element beam of an APERTIF observation we now also want to calibrate and image several beams
# Let's start the pipeline again as we did in the last tutorial
# Remember to change all the directory paths to your own ones
%config IPCompleter.greedy=True
%matplotlib notebook
import lib
lib.setup_logger('info', logfile='/home/adebahr/apercal/ipython-notebooks/tutorials/logs/2.log')
import apercal

root - INFO : Logging started!


root - INFO : To see the log in a bash window use the following command:


root - INFO : tail -n +1 -f /home/adebahr/apercal/ipython-notebooks/tutorials/logs/2.log


Setup logger with lib.setup_logger()


Setup logger with lib.setup_logger()


In [2]:
# Load the config file for the tutorial and look at it
prepare = apercal.prepare('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/2.cfg')
prepare.show(showall=True)

INFO:PREPARE:### Configuration file /home/adebahr/apercal/ipython-notebooks/tutorials/cfg/2.cfg successfully read! ###


INITIAL
	apercaldir = /home/adebahr/apercal
	fluxcal = 3C286.MS
	polcal = 
	target = J1819+3845.MS
	basedir = /data/adebahr/apertif/tutorials/T2/
	beam = 00
	rawsubdir = raw
	crosscalsubdir = crosscal
	selfcalsubdir = selfcal
	linesubdir = line
	contsubdir = continuum
	polsubdir = polarisation
PREPARE
	prepare_obsdir_fluxcal = /data2/apertif/17122114_3C286/
	prepare_obsdir_polcal = 
	prepare_obsdir_target = /data2/apertif/17122115_J1819+3845/
	prepare_obsmode = multi_element
PREFLAG
	preflag_manualflag = True
	preflag_manualflag_fluxcal = True
	preflag_manualflag_polcal = False
	preflag_manualflag_target = True
	preflag_manualflag_auto = True
	preflag_manualflag_antenna = 
	preflag_manualflag_corr = 
	preflag_manualflag_shadow = True
	preflag_manualflag_baseline = 
	preflag_manualflag_channel = 
	preflag_manualflag_time = 17:40:00~20:00:00
	preflag_aoflagger = True
	preflag_aoflagger_bandpass = True
	preflag_aoflagger_fluxcal = True
	preflag_aoflagger_polcal = False
	preflag_aoflagger_

In [3]:
# We are trying to reduce the full FOV of Ger's twinkling quasar (J1819+3845).
# Unfortunately the RA-coordinates for the off-centre beams are still mirrored and the calibration does not work as intended. Anyhow, this notebook can be used as a template as soon as this is fixed.
# We have to set prepare_obsmode = multi_element since these observations were carried out with the full bandwidth in multi-element mode
# The prepare step will copy all 37 beams. You might want to stop the step after the third or fourth copied target dataset.
# The full processing of all beams would need around a day. The pipeline will still work with fewer copied beams.
# Just make sure that all datasets in your directory are correct and not corrupted (most likely you will have to remove the last copied dataset manually since it wasn't copied completely)
prepare.go()

INFO:PREPARE:########## Preparing data for calibration ##########


INFO:PREPARE:### Flux calibrator dataset found. Copying beam 00 to working directory. ###


INFO:PREPARE:### 37 different beams for target field found. ###


INFO:PREPARE:### Copying all target datasets to their working directories. ###


KeyboardInterrupt: 

In [2]:
# As before we have to execute the individual preflag substeps manually
preflag = apercal.preflag('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/2.cfg')

# Add the list of channels to be flagged as in the last tutorial
# Total number of channels
nchannel = 11008

# Make lists of the channels which are affected
a = range(0, nchannel, 64) # the subband edges
b = range(1, nchannel, 64)
c = range(63, nchannel, 64)
d = range(16, nchannel, 64) # the two ghosts
e = range(48, nchannel, 64)
# Combine the channel list into one list and convert to a string
l = a + b + c + d + e
preflag.preflag_manualflag_channel = ';'.join(str(ch) for ch in l)

preflag.show()

PREFLAG
	preflag_manualflag = True
	preflag_manualflag_fluxcal = True
	preflag_manualflag_polcal = False
	preflag_manualflag_target = True
	preflag_manualflag_auto = True
	preflag_manualflag_antenna = 
	preflag_manualflag_corr = 
	preflag_manualflag_shadow = True
	preflag_manualflag_baseline = 
	preflag_manualflag_channel = 0;64;128;192;256;320;384;448;512;576;640;704;768;832;896;960;1024;1088;1152;1216;1280;1344;1408;1472;1536;1600;1664;1728;1792;1856;1920;1984;2048;2112;2176;2240;2304;2368;2432;2496;2560;2624;2688;2752;2816;2880;2944;3008;3072;3136;3200;3264;3328;3392;3456;3520;3584;3648;3712;3776;3840;3904;3968;4032;4096;4160;4224;4288;4352;4416;4480;4544;4608;4672;4736;4800;4864;4928;4992;5056;5120;5184;5248;5312;5376;5440;5504;5568;5632;5696;5760;5824;5888;5952;6016;6080;6144;6208;6272;6336;6400;6464;6528;6592;6656;6720;6784;6848;6912;6976;7040;7104;7168;7232;7296;7360;7424;7488;7552;7616;7680;7744;7808;7872;7936;8000;8064;8128;8192;8256;8320;8384;8448;8512;8576;8640;8704;8768;883

In [3]:
# We want to flag auto-correlations and shadowed antennas.
# The preflag step automatically recognises the other beams and flags them as well using the same pattern.
preflag.manualflag()

In [4]:
# First derive the bandpass for the flagging. This is automatically applied to all target element beams.
preflag.aoflagger_bandpass()

TypeError: a float is required

In [5]:
# Now start aoflagger to flag all the datasets automatically.
preflag.aoflagger_flag()

In [6]:
# Now convert the data to MIRIAD format. Also the CONVERT step takes care of all the different element beams automatically.
convert = apercal.convert('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/2.cfg')
convert.show()
convert.go()

INFO:CONVERT:### Configuration file /home/adebahr/apercal/ipython-notebooks/tutorials/cfg/2.cfg successfully read! ###


INFO:CONVERT:########## FILE CONVERSION started ##########


INFO:CONVERT:### Starting conversion from MS to UVFITS format ###


CONVERT
	convert_ms2uvfits = True
	convert_ms2uvfits_tool_casa_autocorr = False
	convert_uvfits2mir = True
	convert_fluxcal = True
	convert_polcal = False
	convert_target = True


INFO:CONVERT:### Converted MS file 3C286.MS to UVFITS format! ###


...

INFO:CONVERT:### Converted UVFITS file /data/adebahr/apertif/tutorials/T2/00/crosscal/J1819+3845.UVFITS to MIRIAD format! ###


INFO:CONVERT:### Converted UVFITS file /data/adebahr/apertif/tutorials/T2/01/crosscal/J1819+3845.UVFITS to MIRIAD format! ###


INFO:CONVERT:### Converted UVFITS file /data/adebahr/apertif/tutorials/T2/02/crosscal/J1819+3845.UVFITS to MIRIAD format! ###


INFO:CONVERT:### Conversion from UVFITS to MIRIAD format done! ###


INFO:CONVERT:########## FILE CONVERSION done ##########


In [7]:
# We will do the cross calibration now. 
# The pipeline automatically copies the calibrator solutions over to all element beams (crosscal_transfer_to_target = True).
# Even though the cable delay is in place, we still need to solve for delays due to the 5ns sampling (crosscal_delay = True).
ccal = apercal.ccal('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/2.cfg')
ccal.show()
ccal.go()

INFO:CROSSCAL:### Configuration file /home/adebahr/apercal/ipython-notebooks/tutorials/cfg/2.cfg successfully read! ###


INFO:CROSSCAL:########## Starting CROSS CALIBRATION ##########


INFO:CROSSCAL:### Bandpass calibration on the flux calibrator data started ###


CROSSCAL
	crosscal_bandpass = True
	crosscal_delay = True
	crosscal_polarisation = False
	crosscal_transfer_to_target = True


INFO:CROSSCAL:### Bandpass calibration on the flux calibrator data done ###


...

INFO:CROSSCAL:# Calibrator solutions copied to beam 00! #


INFO:CROSSCAL:# Calibrator solutions copied to beam 01! #


INFO:CROSSCAL:# Calibrator solutions copied to beam 02! #


INFO:CROSSCAL:### All solutions copied to target data set(s) ###


INFO:CROSSCAL:########## CROSS CALIBRATION done ##########


In [2]:
# The paramaters for the SELFCAL module are simplified.  I only left the standard reduction in and removed the manual and adaptive ones.
scal = apercal.scal('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/2.cfg')
scal.show()

INFO:SELFCAL:### Configuration file /home/adebahr/apercal/ipython-notebooks/tutorials/cfg/2.cfg successfully read! ###


SELFCAL
	selfcal_image_imsize = 1025
	selfcal_image_cellsize = 6
	selfcal_splitdata = True
	selfcal_splitdata_chunkbandwidth = 0.02
	selfcal_splitdata_channelbandwidth = 0.001
	selfcal_flagantenna = 
	selfcal_flagline = True
	selfcal_flagline_sigma = 0.5
	selfcal_parametric = True
	selfcal_parametric_skymodel_radius = 0.5
	selfcal_parametric_skymodel_cutoff = 0.8
	selfcal_parametric_skymodel_distance = 30
	selfcal_parametric_solint = 5
	selfcal_parametric_uvmin = 0.5
	selfcal_parametric_uvmax = 1000
	selfcal_parametric_amp = False
	selfcal_standard_majorcycle = 3
	selfcal_standard_majorcycle_function = square
	selfcal_standard_minorcycle = 5
	selfcal_standard_minorcycle_function = square
	selfcal_standard_c0 = 10.0
	selfcal_standard_c1 = 5.0
	selfcal_standard_minorcycle0_dr = 3.0
	selfcal_standard_drinit = 25
	selfcal_standard_dr0 = 2.0
	selfcal_standard_nsigma = 50
	selfcal_standard_uvmin = [0.0, 0.0, 0.0]
	selfcal_standard_uvmax = [3000, 3000, 3000]
	selfcal_standard_solint = [5, 3, 

In [3]:
# First you might want to reduce all the beams with the same paramaters. You can easily make a list of the available beams for your dataset:
import glob
from subs import managefiles
# Move to the base directory of the data reduction
managefiles.director(scal,'ch', scal.basedir)
# Get a list of the available beams
beams = glob.glob('*')

print(beams)

INFO:SELFCAL:# Moved to directory /data/adebahr/apertif/tutorials/T2/ #


['00', '01', '05', '17']


In [4]:
# Now iterate over the list of beams and execute the self calibration
for beam in beams:
    scal.beam = beam
    scal.go()

INFO:SELFCAL:########## Starting SELF CALIBRATION ##########


INFO:SELFCAL:# Moved to directory /data/adebahr/apertif/tutorials/T2/00/selfcal #


INFO:SELFCAL:### Splitting of target data into individual frequency chunks started ###


INFO:SELFCAL:# Applying calibrator solutions to target data before averaging #


INFO:SELFCAL:# Calibrator solutions to target data applied #


ERROR:SELFCAL:### No data in your crosscal directory! Exiting pipeline! ###


SystemExit: 1

/home/apercal/pipeline/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# You might want to change the paramaters and redo the calibration for one of the beams individually

# First set the beam you want to redo. For instance 01
scal.beam = '01'
# Then remove the selfcal directory for this beam
scal.reset()
# Then change the paramaters according to your needs
scal.... = ...
# and redo the selfcal without the for loop above. You just want to recalibrate this beam.
scal.go()

In [5]:
# If you are satisfied with the self-calibration of all your beams, we can do the same for the continuum imaging
continuum = apercal.continuum('/home/adebahr/apercal/ipython-notebooks/tutorials/cfg/2.cfg')
continuum.show()

INFO:CONTINUUM:### Configuration file /home/adebahr/apercal/ipython-notebooks/tutorials/cfg/2.cfg successfully read! ###


CONTINUUM
	continuum_image_imsize = 1025
	continuum_image_cellsize = 6
	continuum_image_centre = 
	continuum_image_restorbeam = 
	continuum_image_convolbeam = 45,45,0.0
	continuum_mode = stack
	continuum_majorcycle_function = square
	continuum_minorcycle_function = square
	continuum_minorcycle0_dr = 3.0
	continuum_minorcycle = 5
	continuum_c0 = 10.0
	continuum_c1 = 5.0
	continuum_drinit = 25
	continuum_dr0 = 2.0
	continuum_nsigma = 30
	continuum_robust = 


In [ ]:
# Do continuum imaging for all beams by iterating with the same loop this time over the CONTINUUM module

import glob
from subs import managefiles
# Move to the base directory of the data reduction
managefiles.director(continuum,'ch', continuum.basedir)

beams = glob.glob('*')

for beam in beams:
    continuum.beam = beam
    continuum.go()
    
# You can redo the continuum imaging for individual beams in the same way as for the selfcal module above.
# This holds for all modules working on multiple beams

In [6]:
# Next line imaging. The same as before.

line = apercal.line('/home/adebahr/apercal/ipython-notebooks/commissioning/APERTIF/Lockman_Hole.cfg')
line.show()

INFO:LINE:### Configuration file /home/adebahr/apercal/ipython-notebooks/commissioning/APERTIF/Lockman_Hole.cfg successfully read! ###


LINE
	line_splitdata = True
	line_splitdata_chunkbandwidth = 0.02
	line_splitdata_channelbandwidth = 1.2e-05
	line_transfergains = True
	line_subtract = True
	line_subtract_mode = uvmodel
	line_subtract_mode_uvmodel_majorcycle_function = square
	line_subtract_mode_uvmodel_minorcycle_function = square
	line_subtract_mode_uvmodel_minorcycle = 5
	line_subtract_mode_uvmodel_c0 = 10.0
	line_subtract_mode_uvmodel_c1 = 5.0
	line_subtract_mode_uvmodel_drinit = 50
	line_subtract_mode_uvmodel_dr0 = 3.0
	line_subtract_mode_uvmodel_nsigma = 30
	line_subtract_mode_uvmodel_imsize = 1025
	line_subtract_mode_uvmodel_cellsize = 6
	line_subtract_mode_uvmodel_minorcycle0_dr = 3.0
	line_image = True
	line_image_channels = 
	line_image_imsize = 1025
	line_image_cellsize = 6
	line_image_centre = 
	line_image_robust = -2
	line_image_ratio_limit = 1.2
	line_image_c0 = 10.0
	line_image_c1 = 5.0
	line_image_nsigma = 25
	line_image_minorcycle0_dr = 3.0
	line_image_dr0 = 3.0
	line_image_restorbeam = 
	line_image_

In [ ]:
# And the loop over all the beams

import glob
from subs import managefiles
# Move to the base directory of the data reduction
managefiles.director(line,'ch', line.basedir)

beams = glob.glob('*')

for beam in beams:
    line.beam = beam
    line.go()